In [44]:
with open('16.txt') as f:
    GRID = [line.strip() for line in f.readlines()]

In [45]:
from typing import Literal

Coord = tuple[int, int]
Direction = Literal['>', '<', '^', 'v']
DIRECTIONS: dict[Direction, Coord] = {
    '>': (1, 0),
    '<': (-1, 0),
    '^': (0, -1),
    'v': (0, +1),
}
# we want to go up and right, so we'll prioritise those first
TURNS: dict[Direction, list[tuple[int, Direction]]] = {
    '>': [(1, '>'), (1001, '^'), (1001, 'v')],
    '<': [(1, '<'), (1001, '^'), (1001, 'v')],
    '^': [(1, '^'), (1001, '>'), (1001, '<')],
    'v': [(1, 'v'), (1001, '>'), (1001, '<')],
}

def finds(grid, target):
    for y, line in enumerate(grid):
        for x, char in enumerate(line):
            if char == target:
                return x, y

START = finds(GRID, 'S')
END = finds(GRID, 'E')

In [46]:
def update_costs(
    coord: Coord,
    facing: Direction,
    distances: dict[Coord, list[int]],
    cost: int = 0
):
    x, y = coord
    if GRID[y][x] == '#':
        return distances
    if (d := distances.get(coord)):
        if cost > min(d):
            return distances
    distances.setdefault(coord, []).append(cost)
    dx, dy = DIRECTIONS[facing]
    for c, d in TURNS[facing]:
        dx, dy = DIRECTIONS[d]
        distances = update_costs(
            (x+dx, y+dy), d, distances, cost+c)

    return distances

print(min(update_costs(START, '>', {})[END]))

90460


# Part 2: Pathing

## Attempt 1: Very Naïve, No Good Flood Fill

As it turns out, the `cost < min(d)` requirement is not exhaustive – it excludes alternate paths which still fit the bill.

In [47]:
from functools import reduce
from operator import or_


def get_valid_paths(
    coord: Coord,
    facing: Direction,
    cost: int = 0,
    path: list[Coord] = [],
    distances: dict[Coord, list[int]] = dict(),
):
    x, y = coord
    if GRID[y][x] == '#':
        return
    if GRID[y][x] == 'E':
        yield cost, path
    if coord in path:
        return
    path = path + [coord]
    if (d := distances.get(coord)):
        if cost > min(d):
            return distances
    distances.setdefault(coord, []).append(cost)

    for c, d in TURNS[facing]:
        dx, dy = DIRECTIONS[d]
        yield from get_valid_paths(
            (x+dx, y+dy), d, cost+c, path, distances)

# paths = list(get_valid_paths(START, '>'))
# minscore = min([s for s, _ in paths])
# minpaths = [set(x) for l, x in paths if l == minscore]
# tiles = reduce(or_, minpaths, set())
# grid2 = [list(line) for line in GRID]
# for x, y in tiles:
#     grid2[y][x] = 'O'
# print('\n'.join(''.join(line) for line in grid2))
# print(len(tiles) + 1)

## Attempt 2: Cribbing Future Mia's Work

Hi! Future Mia from Day 18 here. I realised the _A*_ algorithm could work! Typically it involves maintaining a `prev: dict[Coord, Coord]` – a set of arrows to point back the path – but if we loosen it to `prev: dict[Coord, list[Coord]]`, in the occasional instance where a coordinate has the same `cost` from two neighbours, it can reflect that.

Of course, we really mean to represent a `Pointer` – a fusion of coord and direction. This means that we will have to get creative! In particular, if **the same coord but opposite direction** has a known cost, we do not go that way. While technically this might exclude one or two paths, it won't exclude any actual coords.

In [48]:
from queue import Queue

Pointer = tuple[Coord, Direction]

def invert(p: Pointer) -> Pointer:
    match p:
        case (xy, '>'): return (xy, '<')
        case (xy, '<'): return (xy, '>')
        case (xy, 'v'): return (xy, '^')
        case (xy, '^'): return (xy, 'v')

Costs = dict[Pointer | Literal['E'], int]
Prevs = dict[Pointer | Literal['E'], list[Pointer]]

def a_star(grid: list[list[str]], start: Pointer):
    frontier = Queue[Pointer]()
    frontier.put(start)
    costs: Costs = {start: 0}
    prevs: Prevs = {}

    while not frontier.empty():
        p = frontier.get()
        if p == 'E':
            continue
        (x, y), d2 = p
        for c, d2 in TURNS[d2]:
            dx, dy = DIRECTIONS[d2]
            sx, sy = x+dx, y+dy
            if grid[sy][sx] == '#':
                continue
            cost = costs[p] + c
            p2 = (sx, sy), d2
            if invert(p2) in costs:
                continue
            if grid[sy][sx] == 'E':
                p2 = 'E'
            if p2 not in costs or cost < costs[p2]:
                # A fresh path!
                costs[p2] = cost
                prevs[p2] = [p]
                frontier.put(p2)
            elif cost == costs[p2]:
                # A same-cost path - let's record both
                prevs[p2].append(p)
    
    return costs, prevs


costs, prevs = a_star(GRID, (START, '>'))

def path_cells(prevs: Prevs, p: Pointer):
    if p in prevs:
        if p != 'E':
            yield p
        for p2 in prevs[p]:
            yield from path_cells(prevs, p2)

path: list[Pointer] = list(path_cells(prevs, 'E'))
grid2 = [list(line) for line in GRID]
seen: set[Coord] = set()
for (x, y), d in path:
    if (x, y) in seen:
        grid2[y][x] = 'x'
    else:
        grid2[y][x] = d
    seen.add((x, y))
print('\n'.join(''.join(line) for line in grid2).replace('#', ' '))
print(len(seen) + 2)
print(len(costs) ** 0.5)

                                                                                                                                             
 ....... ... . ......................... ....... ... ........... ... ..... ............. ............. ....... ..... ..... ..........^>>>>>E 
 .     . . . . .       .     .   . . . . . .   . . .     .   . .   . .   . . .   .     .     .       . .     . . .   . . .       .   ^     . 
 ..... ... . ......................... . . . . ... ....... ..... ... . . ... ..... ... ............. ... ... ... ..... . ... ... ....^.... . 
 .   .     .       .     . .   .       . . . .             . .   .   . .     .       .       .     . .   . .     .     .   . . . . . ^   . . 
 . . ... . ......... ..... ... . ....... . . . ... ..... ... ..... ... ..... ... ....... ... ................. ......... . ... . . ..^ ... . 
 . .   . .           .       . . .       . . . . . .   . .   .   . .   .   . . . .     . . . . . . .   .     . . .       .     . .   ^     . 
 . ...